# Libraries

In [1]:
%run "/home/cesar/Python_NBs/HDL_Project/HDL_Project/global_fv.ipynb"

User information is ready!


In [2]:
import os
import glob
import pandas as pd
from pandasql import sqldf

# Data

In [3]:
create_new_files = False

## Reading raw files

In [4]:
file_location = '/Restructured_Data/'
files_list = list(glob.iglob('.{}*22.csv'.format(file_location)))
files_list

[]

## Creating reformatted data files

In [5]:
if create_new_files:
    for file_to_edit in files_list:
        # Editing original raw files
        df = pd.read_csv(file_to_edit)

        # datetime is originally structured as DD/MM/YYYY and will be transformed to YYYY/MM/DD
        day = df['date'].str.slice(0, 2)
        month = df['date'].str.slice(3, 5)
        year = df['date'].str.slice(6, 10)
        time = df['date'].str[11:] + ":00"

        df["datetime_edit"] = year + "-" + month + "-" + day + " " + time

        df["date"] = df["datetime_edit"]
        df = df.drop("datetime_edit", axis = 1)    
        df = df.rename(columns={"date": "datetime"})

        file_to_edit = file_to_edit.replace("./", "")

        rename = file_to_edit
        rename = rename.replace(file_location[1:], "")
        rename = rename.replace("_2015_2022.csv", "").lower()
        rename = "sima_" + rename + ".csv"
        rename = file_location[1:] + rename
        
        # Case-specific edit
        rename = rename.replace("2.5", "25")
        
        df.to_csv(rename, encoding='utf-8', index=False)    

# Creating SQL structures

In [6]:
# Reading raw files
file_location = '/Restructured_Data/'
files_list = list(glob.iglob('.{}*sima*.csv'.format(file_location)))

files_list

['./Restructured_Data/sima_co.csv',
 './Restructured_Data/sima_rainf.csv',
 './Restructured_Data/sima_prs.csv',
 './Restructured_Data/sima_tout.csv',
 './Restructured_Data/sima_wsr.csv',
 './Restructured_Data/sima_no.csv',
 './Restructured_Data/sima_wdr.csv',
 './Restructured_Data/sima_nox.csv',
 './Restructured_Data/sima_pm10.csv',
 './Restructured_Data/sima_rh.csv',
 './Restructured_Data/sima_o3.csv',
 './Restructured_Data/sima_so2.csv',
 './Restructured_Data/sima_sr.csv',
 './Restructured_Data/sima_no2.csv',
 './Restructured_Data/sima_pm25.csv']

In [7]:
# Parameters for raw files
files_list = sorted([i[len(file_location)+1:-4] for i in files_list])
data_type = ['F', 'F', 'F', 'F', 'I', 'I', 'I', 'F', 'F', 'I', 'F', 'F', 'I', 'I', 'F']

cols = ['datetime','SE','NE','CE','NO','SO','NO2','NTE','NE2','SE2','SO2','SE3','SUR','NTE2','NE3']

F_types = ['DATETIME NOT NULL', 'FLOAT NULL', 'FLOAT NULL', 'FLOAT NULL', 'FLOAT NULL', 'FLOAT NULL', 'FLOAT NULL', 'FLOAT NULL', 'FLOAT NULL', 'FLOAT NULL', 'FLOAT NULL', 'FLOAT NULL', 'FLOAT NULL', 'FLOAT NULL', 'FLOAT NULL']
I_types = ['DATETIME NOT NULL', 'INT NULL', 'INT NULL', 'INT NULL', 'INT NULL', 'INT NULL', 'INT NULL', 'INT NULL', 'INT NULL', 'INT NULL', 'INT NULL', 'INT NULL', 'INT NULL', 'INT NULL', 'INT NULL']


In [8]:
# Query commands to create SQL tables.
for i in files_list:
    print("CREATE TABLE HDL_Project.`{}` (".format(i))
    for j, k, l in zip(range(len(files_list)), cols, data_type):
        if l == "F":
            dtypes = F_types
        else:
            dtypes = I_types
        
        if j == 0:
            print("`{}` {}".format(k, dtypes[j]))
        else:
            print(", `{}` {}".format(k, dtypes[j]))
    print(") COMMENT = \"Source: {}.csv\";".format(i))
    print("")
        

CREATE TABLE HDL_Project.`sima_co` (
`datetime` DATETIME NOT NULL
, `SE` FLOAT NULL
, `NE` FLOAT NULL
, `CE` FLOAT NULL
, `NO` INT NULL
, `SO` INT NULL
, `NO2` INT NULL
, `NTE` FLOAT NULL
, `NE2` FLOAT NULL
, `SE2` INT NULL
, `SO2` FLOAT NULL
, `SE3` FLOAT NULL
, `SUR` INT NULL
, `NTE2` INT NULL
, `NE3` FLOAT NULL
) COMMENT = "Source: sima_co.csv";

CREATE TABLE HDL_Project.`sima_no` (
`datetime` DATETIME NOT NULL
, `SE` FLOAT NULL
, `NE` FLOAT NULL
, `CE` FLOAT NULL
, `NO` INT NULL
, `SO` INT NULL
, `NO2` INT NULL
, `NTE` FLOAT NULL
, `NE2` FLOAT NULL
, `SE2` INT NULL
, `SO2` FLOAT NULL
, `SE3` FLOAT NULL
, `SUR` INT NULL
, `NTE2` INT NULL
, `NE3` FLOAT NULL
) COMMENT = "Source: sima_no.csv";

CREATE TABLE HDL_Project.`sima_no2` (
`datetime` DATETIME NOT NULL
, `SE` FLOAT NULL
, `NE` FLOAT NULL
, `CE` FLOAT NULL
, `NO` INT NULL
, `SO` INT NULL
, `NO2` INT NULL
, `NTE` FLOAT NULL
, `NE2` FLOAT NULL
, `SE2` INT NULL
, `SO2` FLOAT NULL
, `SE3` FLOAT NULL
, `SUR` INT NULL
, `NTE2` INT NUL

# Command lines to insert contents of csv files into SQL tables

SET GLOBAL local_infile=1;

In [9]:
# Command line. Insert CSV files into created MySQL tables
for i in files_list:
    print("mysql -ucesar -pmysql92@ --local-infile HDL_Project -e \"LOAD DATA LOCAL INFILE \'{}.csv\'  INTO TABLE {} FIELDS TERMINATED BY \',\' LINES TERMINATED BY \'\\n\' IGNORE 1 ROWS\"; ".format(i, i))

mysql -ucesar -pmysql92@ --local-infile HDL_Project -e "LOAD DATA LOCAL INFILE 'sima_co.csv'  INTO TABLE sima_co FIELDS TERMINATED BY ',' LINES TERMINATED BY '\n' IGNORE 1 ROWS"; 
mysql -ucesar -pmysql92@ --local-infile HDL_Project -e "LOAD DATA LOCAL INFILE 'sima_no.csv'  INTO TABLE sima_no FIELDS TERMINATED BY ',' LINES TERMINATED BY '\n' IGNORE 1 ROWS"; 
mysql -ucesar -pmysql92@ --local-infile HDL_Project -e "LOAD DATA LOCAL INFILE 'sima_no2.csv'  INTO TABLE sima_no2 FIELDS TERMINATED BY ',' LINES TERMINATED BY '\n' IGNORE 1 ROWS"; 
mysql -ucesar -pmysql92@ --local-infile HDL_Project -e "LOAD DATA LOCAL INFILE 'sima_nox.csv'  INTO TABLE sima_nox FIELDS TERMINATED BY ',' LINES TERMINATED BY '\n' IGNORE 1 ROWS"; 
mysql -ucesar -pmysql92@ --local-infile HDL_Project -e "LOAD DATA LOCAL INFILE 'sima_o3.csv'  INTO TABLE sima_o3 FIELDS TERMINATED BY ',' LINES TERMINATED BY '\n' IGNORE 1 ROWS"; 
mysql -ucesar -pmysql92@ --local-infile HDL_Project -e "LOAD DATA LOCAL INFILE 'sima_pm10.csv'  

# Backup SQL commands

In [10]:
files_list

['sima_co',
 'sima_no',
 'sima_no2',
 'sima_nox',
 'sima_o3',
 'sima_pm10',
 'sima_pm25',
 'sima_prs',
 'sima_rainf',
 'sima_rh',
 'sima_so2',
 'sima_sr',
 'sima_tout',
 'sima_wdr',
 'sima_wsr']

In [11]:
# DROP TABLE command
for i in files_list:
    print("DROP TABLE HDL_Project.`{}`;".format(i))

DROP TABLE HDL_Project.`sima_co`;
DROP TABLE HDL_Project.`sima_no`;
DROP TABLE HDL_Project.`sima_no2`;
DROP TABLE HDL_Project.`sima_nox`;
DROP TABLE HDL_Project.`sima_o3`;
DROP TABLE HDL_Project.`sima_pm10`;
DROP TABLE HDL_Project.`sima_pm25`;
DROP TABLE HDL_Project.`sima_prs`;
DROP TABLE HDL_Project.`sima_rainf`;
DROP TABLE HDL_Project.`sima_rh`;
DROP TABLE HDL_Project.`sima_so2`;
DROP TABLE HDL_Project.`sima_sr`;
DROP TABLE HDL_Project.`sima_tout`;
DROP TABLE HDL_Project.`sima_wdr`;
DROP TABLE HDL_Project.`sima_wsr`;


# View creation
A view will be created for each station.

In [12]:
col_names = [i[0] for i in qdata("SELECT `station_code` FROM `cat_stations`")]

files_list 

params_list = [i[5:] for i in files_list]

In [13]:
files_list

['sima_co',
 'sima_no',
 'sima_no2',
 'sima_nox',
 'sima_o3',
 'sima_pm10',
 'sima_pm25',
 'sima_prs',
 'sima_rainf',
 'sima_rh',
 'sima_so2',
 'sima_sr',
 'sima_tout',
 'sima_wdr',
 'sima_wsr']

In [14]:
for c in col_names:
    print("CREATE VIEW sima_station_{} AS ".format(c))
    for t, n, k in zip(files_list, params_list, range(len(params_list))):
        if k == 0: 
            select_cols = "Select l{}.datetime, l{}.CE as co".format(k, k)
            table_joins = "from {} l{} ".format(t, k)
        else:
            select_cols = select_cols + ", l{}.{} as {}".format(k, c, n)
            table_joins = table_joins + " left join {} l{} ON l0.datetime = l{}.datetime".format(t, k, k)
            
    print(select_cols) 
    print(table_joins + ";")
    print("")

CREATE VIEW sima_station_SE AS 
Select l0.datetime, l0.CE as co, l1.SE as no, l2.SE as no2, l3.SE as nox, l4.SE as o3, l5.SE as pm10, l6.SE as pm25, l7.SE as prs, l8.SE as rainf, l9.SE as rh, l10.SE as so2, l11.SE as sr, l12.SE as tout, l13.SE as wdr, l14.SE as wsr
from sima_co l0  left join sima_no l1 ON l0.datetime = l1.datetime left join sima_no2 l2 ON l0.datetime = l2.datetime left join sima_nox l3 ON l0.datetime = l3.datetime left join sima_o3 l4 ON l0.datetime = l4.datetime left join sima_pm10 l5 ON l0.datetime = l5.datetime left join sima_pm25 l6 ON l0.datetime = l6.datetime left join sima_prs l7 ON l0.datetime = l7.datetime left join sima_rainf l8 ON l0.datetime = l8.datetime left join sima_rh l9 ON l0.datetime = l9.datetime left join sima_so2 l10 ON l0.datetime = l10.datetime left join sima_sr l11 ON l0.datetime = l11.datetime left join sima_tout l12 ON l0.datetime = l12.datetime left join sima_wdr l13 ON l0.datetime = l13.datetime left join sima_wsr l14 ON l0.datetime = l14.d

# Data Understanding
After inserting information to SQL tables

## How many days have less than 24 hours?
There are two scenarios:
* If there are any, then the date sequence is incomplete.
* If the query returns empty, then the sequence is complete


In [15]:
for i in files_list:
    sqlq = """
    select * from (
    select year, month, day, count(*) as conteo from (
        SELECT *, year(datetime) as `year`, month(datetime) as `month`, day(datetime) as `day`, hour(datetime) as `hour` FROM `{}`
    ) s1
    GROUP by year, month, day
    ) s2
    where conteo <24
    """.format(i)
    days_evaluation = qdata(sqlq)
    
    if days_evaluation:
        print("{}".format(i))
        print("Days with less than 24H: {}".format(days_evaluation))
        print("")

sima_no
Days with less than 24H: [(2015, 8, 1, 1), (2015, 8, 2, 1), (2015, 8, 3, 1), (2015, 8, 4, 1), (2015, 8, 5, 1), (2015, 8, 6, 1), (2015, 8, 7, 1), (2015, 8, 8, 1), (2015, 8, 9, 1), (2015, 8, 10, 1), (2015, 8, 11, 1), (2015, 8, 12, 1), (2015, 8, 13, 1), (2015, 8, 14, 1), (2015, 8, 15, 1), (2015, 8, 16, 1), (2015, 8, 17, 1), (2015, 8, 18, 1), (2015, 8, 19, 1), (2015, 8, 20, 1), (2015, 8, 21, 1), (2015, 8, 22, 1), (2015, 8, 23, 1), (2015, 8, 24, 1), (2015, 8, 25, 1), (2015, 8, 26, 1), (2015, 8, 27, 1), (2015, 8, 28, 1), (2015, 8, 29, 1), (2015, 8, 30, 1), (2015, 8, 31, 1), (2015, 9, 1, 1), (2015, 9, 2, 1), (2015, 9, 3, 1), (2015, 9, 4, 1), (2015, 9, 5, 1), (2015, 9, 6, 1), (2015, 9, 7, 1), (2015, 9, 8, 1), (2015, 9, 9, 1), (2015, 9, 10, 1), (2015, 9, 11, 1), (2015, 9, 12, 1), (2015, 9, 13, 1), (2015, 9, 14, 1), (2015, 9, 15, 1), (2015, 9, 16, 1), (2015, 9, 17, 1), (2015, 9, 18, 1), (2015, 9, 19, 1), (2015, 9, 20, 1), (2015, 9, 21, 1), (2015, 9, 22, 1), (2015, 9, 23, 1), (2015, 9, 24

## How many days are per year-month

In [16]:
for i in files_list:
    sqlq = """
    select count(*) from (
    select year, month, count(*)/24 as conteo from (
        SELECT *, year(datetime) as `year`, month(datetime) as `month`, day(datetime) as `day`, hour(datetime) as `hour` FROM `{}`
    ) s1
    GROUP by year, month
    ) s2
    """.format(i)
    
    print("{}".format(i))
    print(qdata(sqlq))
    print("")

sima_co
[(88,)]

sima_no
[(89,)]

sima_no2
[(88,)]

sima_nox
[(89,)]

sima_o3
[(88,)]

sima_pm10
[(88,)]

sima_pm25
[(88,)]

sima_prs
[(88,)]

sima_rainf
[(88,)]

sima_rh
[(88,)]

sima_so2
[(88,)]

sima_sr
[(88,)]

sima_tout
[(88,)]

sima_wdr
[(88,)]

sima_wsr
[(88,)]

